### 从.mat文件中读入数据

In [2]:
import scipy
import numpy as np

In [3]:
data=scipy.io.loadmat('DBPB.mat')#读入由matlab处理好的样本特征向量

In [4]:
data['DBPB1'][0].shape#每个样本94个特征

(94,)

In [5]:
data['DBPB1'].shape#第一类样本数量为9279

(9279, 94)

### 先将数据读入到矩阵中便于处理

In [6]:
data['DBPB1']

array([[0.00172604, 0.00379138, 0.00296589, ..., 0.00116799, 0.00091695,
        0.00525093],
       [0.00352267, 0.00097731, 0.00296935, ..., 0.00847652, 0.00462204,
        0.00282324],
       [0.00339284, 0.00166496, 0.00277194, ..., 0.00288942, 0.00499452,
        0.00355836],
       ...,
       [0.00799746, 0.00392692, 0.00227704, ..., 0.00519247, 0.00426208,
        0.00183095],
       [0.00149238, 0.00276466, 0.00239477, ..., 0.00100603, 0.00415174,
        0.00158   ],
       [0.00478357, 0.00589117, 0.006869  , ..., 0.00381122, 0.01051007,
        0.01011765]])

In [9]:
data_1=np.array(data['DBPB1'])
data_2=np.array(data['DBPB2'])

发现.mat读入的数据就是np类型数组，待定处理

### 接下来进行数据平衡

In [8]:
from imblearn.over_sampling import SMOTE,ADASYN

In [10]:
ada=ADASYN(random_state=42)

In [14]:
x=np.concatenate((data_1,data_2),axis=0)
x.shape

(9989, 94)

In [19]:
data_1_y=np.zeros(data_1.shape[0])
data_2_y=np.full(data_2.shape[0],1)

In [23]:
y=np.concatenate((data_1_y,data_2_y),axis=0)

In [24]:
y.shape

(9989,)

In [25]:
x_1,y_1=ada.fit_resample(x,y)

In [26]:
x_1.shape

(18481, 94)

In [27]:
y_1.shape

(18481,)

In [31]:
sum(y_1==1)

9202